In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MovieRecommender") \
    .master("local[*]") \
    .config("spark.jars", "postgresql-42.7.1.jar") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://localhost:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

# Load tables
ratings = spark.read.jdbc(jdbc_url, "movielens.ratings", properties=properties)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

ratings.show(5)

26/01/24 18:47:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
[Stage 0:>                                                          (0 + 1) / 1]

+-------+--------+------+-------------------+
|user_id|movie_id|rating|   rating_timestamp|
+-------+--------+------+-------------------+
|  83341|    2985|   2.0|1999-11-16 13:13:17|
|  83341|    2987|   4.0|1999-11-15 13:04:54|
|  83341|    3033|   2.0|1999-11-16 12:33:01|
|  83341|    3039|   4.0|1999-11-16 12:30:00|
|  83341|    3041|   1.0|1999-11-16 12:35:46|
+-------+--------+------+-------------------+
only showing top 5 rows


In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Prepare data
(training, test) = ratings.randomSplit([0.8, 0.2], seed=42)

# Build ALS model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop"
)

model = als.fit(training)

# Evaluate
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

26/01/24 18:48:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/01/24 18:48:04 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [ ]:
# Top 10 recommendations for each user
user_recs = model.recommendForAllUsers(10)
user_recs.show(5, truncate=False)

# Top 10 recommendations for a specific user
user_42_recs = model.recommendForUserSubset(
    spark.createDataFrame([(42,)], ["user_id"]), 10
)

# Join with movie titles
from pyspark.sql.functions import explode

user_42_recs_flat = user_42_recs.select(
    "user_id",
    explode("recommendations").alias("rec")
).select("user_id", "rec.movie_id", "rec.rating")

user_42_recs_flat.join(movies, "movie_id").show(truncate=False)

In [ ]:
from pyspark.sql.functions import explode, col

movie_recs = model.recommendForItemSubset(
    spark.createDataFrame([(1,)], ["movie_id"]), 10
)

# Check the schema
movie_recs.printSchema()

# Explode correctly - it's user_id, not movie_id
movie_recs_flat = movie_recs.select(
    col("movie_id"),
    explode("recommendations").alias("rec")
).select(
    "movie_id",
    col("rec.user_id").alias("user_id"),
    col("rec.rating").alias("score")
)

movie_recs_flat.show()

In [ ]:
from pyspark.ml.feature import Normalizer
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors
import numpy as np

# Get item factors
item_factors = model.itemFactors

# Get Toy Story's factor vector
toy_story_factor = item_factors.filter(col("id") == 1).select("features").collect()[0][0]

# Calculate cosine similarity
def cosine_sim(v):
    a = np.array(toy_story_factor)
    b = np.array(v)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

cosine_udf = udf(cosine_sim, DoubleType())

# Find similar movies
similar_movies = item_factors \
    .withColumn("similarity", cosine_udf(col("features"))) \
    .filter(col("id") != 1) \
    .orderBy(col("similarity").desc()) \
    .limit(10) \
    .withColumnRenamed("id", "movie_id")

# Join with movie titles
similar_movies.join(movies, "movie_id") \
    .select("movie_id", "title", "similarity") \
    .show(truncate=False)